In [2]:
import requests
import time
import random
import re
from openpyxl import Workbook
from bs4 import BeautifulSoup
from urllib.request import Request
from urllib.request import urlopen

UAS = ("Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1", 
       "Mozilla/5.0 (Windows NT 6.3; rv:36.0) Gecko/20100101 Firefox/36.0",
       "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10; rv:33.0) Gecko/20100101 Firefox/33.0",
       "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
       "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36",
       "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36",
       )

ua = UAS[random.randrange(len(UAS))]
headers = {'user-agent': ua}

In [27]:
def get_urls(url):
    """从每个列表页获取文章url"""
    req = Request(url= url, headers = headers)
    html = urlopen(req).read()
    soup = BeautifulSoup(html, features='lxml')
    all_href= soup.find_all('a', class_='btn')
    b_list = []
    for l in all_href:
        b_list.append('https://askderm.com'+ l['href'])
    return b_list

In [28]:
get_urls('https://askderm.com/blogs/askderm-blog?page=1')

['https://askderm.com/blogs/askderm-blog/2017-holiday-gift-guide',
 'https://askderm.com/blogs/askderm-blog/nuface-nubody-giveaway',
 'https://askderm.com/blogs/askderm-blog/how-to-slay-the-4th-of-july',
 'https://askderm.com/blogs/askderm-blog/glowbiotics-brightening-essential-kit-giveaway',
 'https://askderm.com/blogs/askderm-blog/beauty-chemistry-101-what-to-mix-and-what-not-to-mix']

In [33]:
def get_info(url):
    """获取标题、作者、发布时间及正文内容"""
    time.sleep(0.1)
    req = Request(url, headers = headers)
    html = urlopen(req).read()
    soup = BeautifulSoup(html, features='lxml')
  
    if soup.find('h1') == None:
        return {'url': url, 'title': None, 'author and date': None, 'body': None}
    else:
        title = soup.find('h1').text
        link = soup.find('p',class_='entry-meta')
        author= link.find('strong').text
        date=link.find('time').text
        author_date = author + ' '+ date
        p_list=[]
        for p in soup.find_all('div', itemprop='articleBody'):
            p_list.append(p.text)
            body = '\n'.join(p_list)
        return {'url': url, 'title': title, 'author and date': author_date, 'body': body}

In [34]:
get_info('https://askderm.com/blogs/askderm-blog/glow-like-a-holiday-star')

{'url': 'https://askderm.com/blogs/askderm-blog/glow-like-a-holiday-star',
 'title': 'Glow Like a Holiday Star',
 'author and date': 'Vanessa Sterling December 03, 2017',
 'body': "\nThe holiday season is all about shiny ornaments, shimmering light displays, warm glowing fireplaces…but one thing that tends to not glow during the holidays is dull chapped and dry skin! No matter how perfect your skin is, we all seem to get more sensitive and dry during the holiday season. The key to beating the cold and dry weather is exfoliation, nourishing and protecting your delicate winter skin, and askderm.com has the perfect recipe to looking your best for every holiday occasion.\nBe Gentle\nThe most significant mistake that we can fall into with our skin is using a cleansing regimen that is too harsh. DCL Express Waterless Makeup Remover is gentle yet effective and won’t dry out your skin. It also has calming ingredients like Aloe and Chamomile and uses Hyaluronic Acid to keep moisture attracted t

In [35]:
def save_as_xls(info_list):
    """将获取到的文章信息存储到本地"""
    workbook = Workbook()
    worksheet = workbook.active
    worksheet.title = 'sheet1'
    titles = ['URL', 'Title', 'Author and Date', 'Content']
    keys = ['url', 'title', 'author and date', 'body']
    for index, title in enumerate(titles, start=1):
        worksheet.cell(row=1, column=index, value=title)
    for i, info in enumerate(info_list, start=1):
        for j,key in enumerate(keys, start=1):
            worksheet.cell(row=i+1, column=j, value=info[key])
    workbook.save('info.xlsx')

In [36]:
list=[]
for i in range(1,27):
    list.append('https://askderm.com/blogs/askderm-blog?page='+ str(i))
list

['https://askderm.com/blogs/askderm-blog?page=1',
 'https://askderm.com/blogs/askderm-blog?page=2',
 'https://askderm.com/blogs/askderm-blog?page=3',
 'https://askderm.com/blogs/askderm-blog?page=4',
 'https://askderm.com/blogs/askderm-blog?page=5',
 'https://askderm.com/blogs/askderm-blog?page=6',
 'https://askderm.com/blogs/askderm-blog?page=7',
 'https://askderm.com/blogs/askderm-blog?page=8',
 'https://askderm.com/blogs/askderm-blog?page=9',
 'https://askderm.com/blogs/askderm-blog?page=10',
 'https://askderm.com/blogs/askderm-blog?page=11',
 'https://askderm.com/blogs/askderm-blog?page=12',
 'https://askderm.com/blogs/askderm-blog?page=13',
 'https://askderm.com/blogs/askderm-blog?page=14',
 'https://askderm.com/blogs/askderm-blog?page=15',
 'https://askderm.com/blogs/askderm-blog?page=16',
 'https://askderm.com/blogs/askderm-blog?page=17',
 'https://askderm.com/blogs/askderm-blog?page=18',
 'https://askderm.com/blogs/askderm-blog?page=19',
 'https://askderm.com/blogs/askderm-blog

In [37]:
def main():
    """main function"""
    info_list = []
    for i in list:
        b_list = get_urls(i)
        for url in b_list:
            info_list.append(get_info(url))
    save_as_xls(info_list)
    
if __name__ == "__main__":
    main()

In [20]:
get_info('https://askderm.com/blogs/askderm-blog/beauty-chemistry-101-what-to-mix-and-what-not-to-mix')

{'url': 'https://askderm.com/blogs/askderm-blog/beauty-chemistry-101-what-to-mix-and-what-not-to-mix',
 'title': 'Beauty Chemistry 101: What to Mix and What Not to Mix',
 'author and date': 'Vanessa Sterling June 04, 2018',
 'body': ' Using these two ingredients in tandem can lead to extreme drying, flaking and irritation which can ultimately lead to breakouts and skin that looks more aged. Retinols/Retinoids usually take about 10 days or more to leave your system so if you want to be able to get the benefits of both ingredients its better to start with your salicylic or glycolic regimen until you are satisfied with your results, discontinue your use and then start with your retinol/retinoid routine. If you have extremely oily skin and you find that your skin can handle using products with both of these ingredients at the same time stick to a cleanser with salicylic or glycolic acid because it has shorter contact with your skin than a cream ointment or gel. \nStart with the Murad Acne 

In [18]:
url = 'https://askderm.com/blogs/askderm-blog/beauty-chemistry-101-what-to-mix-and-what-not-to-mix'
req = Request(url, headers = headers)
html = urlopen(req).read()
soup = BeautifulSoup(html, features='lxml')   
title = soup.find('h1').text
for link in soup.find_all('p',class_='entry-meta'):
        author= link.find('strong').text
        date=link.find('time').text
author_date = author + ' '+ date
p_list=[]
for p in soup.find_all('span', class_='s1'):
    p_list.append(p.text)
body = '\n'.join(p_list)
print(title)

Beauty Chemistry 101: What to Mix and What Not to Mix


In [6]:
get_urls('https://askderm.com/blogs/askderm-blog?page=1')

['/blogs/askderm-blog/2017-holiday-gift-guide',
 '/blogs/askderm-blog/nuface-nubody-giveaway',
 '/blogs/askderm-blog/how-to-slay-the-4th-of-july',
 '/blogs/askderm-blog/glowbiotics-brightening-essential-kit-giveaway',
 '/blogs/askderm-blog/beauty-chemistry-101-what-to-mix-and-what-not-to-mix']